In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations
import itertools

In [2]:
# ====================================================
# Get the difference
# ====================================================
def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1


In [3]:
'''train = pd.read_parquet('../input/train.parquet')
test = pd.read_parquet('../input/test.parquet')'''

"train = pd.read_parquet('../input/train.parquet')\ntest = pd.read_parquet('../input/test.parquet')"

In [4]:
'''# Delete columns with missing values above threshold:
def feature_filter(df, threshold=0.95):
    filtered_features = [col for col in df.columns if df[col].isnull().sum()/len(df) < threshold]
    return filtered_features'''

'# Delete columns with missing values above threshold:\ndef feature_filter(df, threshold=0.95):\n    filtered_features = [col for col in df.columns if df[col].isnull().sum()/len(df) < threshold]\n    return filtered_features'

In [5]:
'''train = train[feature_filter(train)]
test = test[feature_filter(test)]'''

'train = train[feature_filter(train)]\ntest = test[feature_filter(test)]'

In [6]:
'''# Replace null values with appropriate entity:
train["P_2"].fillna(1.05, inplace = True)
train["P_3"].fillna(2.45, inplace = True)'''

'# Replace null values with appropriate entity:\ntrain["P_2"].fillna(1.05, inplace = True)\ntrain["P_3"].fillna(2.45, inplace = True)'

In [7]:
'''# Handpicked feature Engineering:
def handpicked(df):
    df['payment_default'] = df['P_2'] * df['D_39']
    df['payment'] = df.P_2 * df.P_3 * df.P_4
    temp = df['D_39'].value_counts().to_dict()
    df['D_39_counts'] = df['D_39'].map(temp)
    return df
handpicked(train)
handpicked(test)
print(len(train.columns), len(test.columns))'''

"# Handpicked feature Engineering:\ndef handpicked(df):\n    df['payment_default'] = df['P_2'] * df['D_39']\n    df['payment'] = df.P_2 * df.P_3 * df.P_4\n    temp = df['D_39'].value_counts().to_dict()\n    df['D_39_counts'] = df['D_39'].map(temp)\n    return df\nhandpicked(train)\nhandpicked(test)\nprint(len(train.columns), len(test.columns))"

In [8]:
'''#compute after-pay feature:
def after_pay(df):
    for bcol in [f'B_{i}' for i in [11,14,17]]+['D_39','D_131']+[f'S_{i}' for i in [16,23]]:
        for pcol in ['P_2','P_3']:
            if bcol in train.columns:
                df[f'{bcol}-{pcol}'] = df[bcol] - df[pcol]
after_pay(train)
after_pay(test)
print(len(train.columns), len(test.columns))'''

"#compute after-pay feature:\ndef after_pay(df):\n    for bcol in [f'B_{i}' for i in [11,14,17]]+['D_39','D_131']+[f'S_{i}' for i in [16,23]]:\n        for pcol in ['P_2','P_3']:\n            if bcol in train.columns:\n                df[f'{bcol}-{pcol}'] = df[bcol] - df[pcol]\nafter_pay(train)\nafter_pay(test)\nprint(len(train.columns), len(test.columns))"

In [9]:
'''train.to_parquet('../input/train_smn_2.parquet')
test.to_parquet('../input/test_smn_2.parquet')'''

"train.to_parquet('../input/train_smn_2.parquet')\ntest.to_parquet('../input/test_smn_2.parquet')"

In [10]:
# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    train = pd.read_parquet('../input/train_smn_2.parquet')
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    train_labels = pd.read_csv('../input/train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)
    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_diff
    gc.collect()
    test = pd.read_parquet('../input/test_smn_2.parquet')
    print('Starting test feature engineer...')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    # Transform float64 columns to float32
    cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        test_num_agg[col] = test_num_agg[col].astype(np.float32)
    # Transform int64 columns to int32
    cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        test_cat_agg[col] = test_cat_agg[col].astype(np.int32)
    # Get the difference
    test_diff = get_difference(test, num_features)
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')
    del test_num_agg, test_cat_agg, test_diff
    gc.collect()
    # Save files to disk
    train.to_parquet('../input/train_fe_smn_2.parquet')
    test.to_parquet('../input/test_fe_smn_2.parquet')

# Read & Preprocess Data
#read_preprocess_data()

In [11]:
# ====================================================
# Directory settings
# ====================================================
import os

expt_name = "lgbm_smn_5_fold_6"
OUTPUT_DIR = f'../{expt_name}/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)



In [12]:

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '../input/'
    seed = 42
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


In [13]:
# ====================================================
# Read data
# ====================================================
train = pd.read_parquet(CFG.input_dir + 'train_fe_smn_2.parquet')
test = pd.read_parquet(CFG.input_dir + 'test_fe_smn_2.parquet')

In [14]:
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,...,D_131-P_2_min,D_131-P_2_max,D_131-P_2_last,D_131-P_3_mean,D_131-P_3_std,D_131-P_3_min,D_131-P_3_max,D_131-P_3_last,S_16-P_2_mean,S_16-P_2_std,S_16-P_2_min,S_16-P_2_max,S_16-P_2_last,S_16-P_3_mean,S_16-P_3_std,S_16-P_3_min,S_16-P_3_max,S_16-P_3_last,S_23-P_2_mean,S_23-P_2_std,S_23-P_2_min,S_23-P_2_max,S_23-P_2_last,S_23-P_3_mean,S_23-P_3_std,S_23-P_3_min,S_23-P_3_max,S_23-P_3_last,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,P_2_diff1,D_39_diff1,B_1_diff1,B_2_diff1,R_1_diff1,S_3_diff1,D_41_diff1,B_3_diff1,D_42_diff1,D_43_diff1,D_44_diff1,B_4_diff1,D_45_diff1,B_5_diff1,R_2_diff1,D_46_diff1,D_47_diff1,D_48_diff1,D_49_diff1,B_6_diff1,B_7_diff1,B_8_diff1,D_50_diff1,D_51_diff1,B_9_diff1,R_3_diff1,D_52_diff1,P_3_diff1,B_10_diff1,D_53_diff1,S_5_diff1,B_11_diff1,S_6_diff1,D_54_diff1,R_4_diff1,S_7_diff1,B_12_diff1,S_8_diff1,D_55_diff1,D_56_diff1,B_13_diff1,R_5_diff1,D_58_diff1,S_9_diff1,B_14_diff1,D_59_diff1,D_60_diff1,D_61_diff1,B_15_diff1,S_11_diff1,D_62_diff1,D_65_diff1,B_16_diff1,B_17_diff1,B_18_diff1,B_19_diff1,B_20_diff1,S_12_diff1,R_6_diff1,S_13_diff1,B_21_diff1,D_69_diff1,B_22_diff1,D_70_diff1,D_71_diff1,D_72_diff1,S_15_diff1,B_23_diff1,P_4_diff1,D_74_diff1,D_75_diff1,D_76_diff1,B_24_diff1,R_7_diff1,D_77_diff1,B_25_diff1,B_26_diff1,D_78_diff1,D_79_diff1,R_8_diff1,R_9_diff1,S_16_diff1,D_80_diff1,R_10_diff1,R_11_diff1,B_27_diff1,D_81_diff1,D_82_diff1,S_17_diff1,R_12_diff1,B_28_diff1,R_13_di

In [15]:
len(train.columns)

1163

In [16]:


# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    # Get the difference between last and mean
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
    # Transform float64 and float32 to float16
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': CFG.metric,
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        #'path_smooth': 20, #added new
        'min_data_in_leaf': 40, #changed from 40
        'device': 'gpu',
        'gpu_platform_id': 1,
        'gpu_device_id': 0
        }
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 15000,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 1500,
            verbose_eval = 500,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, OUTPUT_DIR+f'lgbm_fold{fold}_seed{CFG.seed}.pkl')
        # Predict validation
        val_pred = model.predict(x_val)
        # Add to out of folds array
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        test_pred = model.predict(test[features])
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(OUTPUT_DIR+f'oof_{expt_name}_seed{CFG.seed}.csv', index = False)
    # Create a dataframe to store test prediction
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(OUTPUT_DIR+f'test_{expt_name}_seed{CFG.seed}.csv', index = False)


seed_everything(CFG.seed)
#train, test = read_data()
train_and_evaluate(train, test)

  0%|          | 0/1164 [00:00<?, ?it/s]

 
--------------------------------------------------
Training fold 0 with 1452 features...
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 246946
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 1444
[LightGBM] [Info] Using requested OpenCL platform 1 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 847 dense feature groups (296.90 MB) transferred to GPU in 0.133886 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
[500]	training's binary_logloss: 0.334721	training's amex_metric: 0.779609	valid_1's binary_logloss: 0.3366